# 2B Модель

In [32]:
import chromadb
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma

# 🔹 Загружаем модель 
model_name = "google/gemma-2b-it" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# 🔹 Создаем пайплайн для инференса
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


# Подгружаем 8B модель

In [1]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quant_config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             quantization_config=quant_config,
                                             device_map="auto")

# Создаем пайплайн для модели
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
import json

def extract_location(query: str):
    """Извлекает страны и города с помощью LLM в формате few-shot."""

    prompt = f"""Извлеки названия стран и городов из запроса. Ответ строго в формате JSON.

    Пример 1:
    Текст: "Какие достопримечательности есть в Москве и Париже?"
    Ответ:
    {{
      "cities": ["Москва", "Париж"],
      "countries": ["Россия", "Франция"]
    }}
    
    Пример 2:
    Текст: "Что посмотреть в Берлине, Лондоне и Италии?"
    Ответ:
    {{
      "cities": ["Берлин", "Лондон"],
      "countries": ["Германия", "Великобритания", "Италия"]
    }}
    
    Пример 3:
    Текст: "{query}"
    Ответ:
    """

    response = llm_pipeline(prompt, max_new_tokens=100, do_sample=False)
    response_text = response[0]["generated_text"]

    try:
        # Разделим на части по слову 'Ответ' и возьмём сгенерированный ответ
        parts = response_text.split("Ответ:")
        answer_part = parts[3] 

        # Найдём первую подстроку от "{" до "}" 
        json_start = answer_part.find("{")
        json_end = answer_part.find("}") + 2

        if json_start == -1 or json_end == -1:
            raise ValueError("Не удалось найти JSON-блок")

        json_str = answer_part[json_start:json_end]
        result = json.loads(json_str)

    except Exception as e:
        print(f"Ошибка извлечения: {e}")
        result = {"cities": [], "countries": []}

    return result

#user_query = "Что можно посмотреть в Казани и в Германии?"
#print(extract_location(user_query))

# Тестирование ретривера


```
Предполагается, что модель сможет вычленять названия стран и городов из вопросов пользователей, соответственно мы можем оценить ретривер путем оценки соответствия извлеченных сущностей запросу. Тк в дальнейшем мы будем фильтровать чанки по мета информации (Страна, город) в контекст модели не будет попадать нерелевантная информация
```

## Тестовые запросы и правильные ответы

In [47]:
test_queries = [
    "Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?",
    "Что лучше попробовать из еды в Париже и Лионе?",
    "Какие музеи есть в Берлине и Мюнхене?",
    "Где можно покататься на гондолах в Италии?",
    "Какой пляжный отдых можно найти во Флоренции или Венеции?",
    "Какой общественный транспорт лучше использовать в Риме?",
    "Какова история Красной площади в Москве?",
    "Какие замки можно посетить во Франции?",
    "Какие современные здания есть в Берлине?",
    "Что стоит посетить в Казани туристу впервые?"
]

expected_entities = [
    {"cities": ["Москва", "Санкт-Петербург"], "countries": ["Россия"]},
    {"cities": ["Париж", "Лион"], "countries": ["Франция"]},
    {"cities": ["Берлин", "Мюнхен"], "countries": ["Германия"]},
    {"cities": [], "countries": ["Италия"]},  # Вопрос без конкретных городов
    {"cities": ["Флоренция", "Венеция"], "countries": ["Италия"]},
    {"cities": ["Рим"], "countries": ["Италия"]},
    {"cities": ["Москва"], "countries": ["Россия"]},
    {"cities": [], "countries": ["Франция"]},
    {"cities": ["Берлин"], "countries": ["Германия"]},
    {"cities": ["Казань"], "countries": ["Россия"]}
]

## Метрики оценки извлечения сущностей

### 1. Full Match Accuracy 

Эта метрика показывает, насколько точно модель извлекает все нужные сущности (города и страны).

**Как считается**:

Для каждого запроса проверяется, совпадают ли оба списка — и стран, и городов — в точности с ожидаемыми значениями:

Если совпадает — считаем 1 балл.
Если хотя бы одна лишняя или пропущенная сущность — 0 баллов.

Затем подсчитывается доля таких полностью верных предсказаний среди всех запросов.


### 2. IoU (Intersection over Union)

Метрика оценивает пересечение между предсказанными и ожидаемыми сущностями — то есть насколько сильно они совпадают, даже если не идеально.

**Как считается**:

Сначала отдельно для городов и отдельно для стран:

Считается, сколько сущностей было предсказано верно.
Сравнивается это с общим числом всех уникальных сущностей в предсказании и эталоне (то есть объединением).
Полученные значения объединяются в одно среднее число.

Затем IoU усредняется по всем запросам


In [4]:
from typing import List, Dict

def evaluate_retriever(predicted_entities: List[Dict],
                       queries: List[str],
                       expected: List[Dict]):
    """Оценивает точность выделения стран и городов альтернативными метриками."""

    correct_full_predictions = 0
    total_iou = 0

    for i, (pred, true) in enumerate(zip(predicted_entities, expected)):
        # Удаляем дубликаты
        pred_cities = set(pred.get("cities", []))
        pred_countries = set(pred.get("countries", []))

        true_cities = set(true.get("cities", []))
        true_countries = set(true.get("countries", []))

        # Проверка на точное совпадение
        if pred_cities == true_cities and pred_countries == true_countries:
            correct_full_predictions += 1

        # IoU по городам
        union_cities = pred_cities | true_cities
        inter_cities = pred_cities & true_cities
        cities_iou = len(inter_cities) / len(union_cities) if union_cities else 1.0

        # IoU по странам
        union_countries = pred_countries | true_countries
        inter_countries = pred_countries & true_countries
        countries_iou = len(inter_countries) / len(union_countries) if union_countries else 1.0

        avg_iou = (cities_iou + countries_iou) / 2
        total_iou += avg_iou

        print(f"\nВопрос {i+1}: {queries[i]}")
        print(f"Ожидалось: {true}")
        print(f"Предсказано: {pred}")
        print(f"IoU: {avg_iou:.2f}")
        print(f"Точное совпадение: {int(pred_cities == true_cities and pred_countries == true_countries)}")

    print("\n**Общая оценка**")
    print(f"Полностью верных предсказаний: {correct_full_predictions / len(queries):.2f}")
    print(f"Средний IoU: {total_iou / len(queries):.2f}")


In [49]:
# Инференсим модель и сохраняем ответы
predicted_entities = [extract_location(query) for query in test_queries]

In [50]:
predicted_entities

[{'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']},
 {'cities': ['Париж', 'Лион'], 'countries': ['Франция']},
 {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']},
 {'cities': ['Италия'], 'countries': ['Италия']},
 {'cities': ['Флоренция', 'Венеция'], 'countries': ['Италия']},
 {'cities': ['Рим'], 'countries': ['Италия']},
 {'cities': ['Москва'], 'countries': ['Россия']},
 {'cities': [], 'countries': ['Франция']},
 {'cities': ['Берлин'], 'countries': ['Германия']},
 {'cities': ['Казань'], 'countries': ['Россия']}]

In [63]:
evaluate_retriever(predicted_entities, test_queries, expected_entities)


Вопрос 1: Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?
Ожидалось: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия']}
Предсказано: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 2: Что лучше попробовать из еды в Париже и Лионе?
Ожидалось: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
Предсказано: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
IoU: 1.00
Точное совпадение: 1

Вопрос 3: Какие музеи есть в Берлине и Мюнхене?
Ожидалось: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']}
Предсказано: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 4: Где можно покататься на гондолах в Италии?
Ожидалось: {'cities': [], 'countries': ['Италия']}
Предсказано: {'cities': ['Италия'], 'countries': ['Италия']}
IoU: 0.50
Точное совпадение: 0

Вопрос 5: Какой пляжный отдых можно найти во Флоренции или 